In [ ]:
%pip install qiskit -U
%pip install qiskit-ibm-runtime -U

In [ ]:
import qiskit_ibm_runtime
print(qiskit_ibm_runtime.version.get_version_info())

# Real quantum computer

## [Background on the various packages](https://github.com/Qiskit/qiskit-ibm-provider/blob/main/docs/tutorials/Migration_Guide_from_qiskit-ibmq-provider.ipynb)
- [Qiskit IBM Runtime](https://github.com/Qiskit/qiskit-ibm-runtime) - [pypi](https://pypi.org/project/qiskit-ibm-runtime) - Qiskit Runtime is a new architecture offered by IBM Quantum that streamlines quantum computations. It is designed to use classical compute resources to execute quantum circuits with more efficiency on quantum processors.
- [Qiskit IBM Provider](https://github.com/Qiskit/qiskit-ibm-provider) - [pypi](https://pypi.org/project/qiskit-ibm-provider) - This project contains a provider that allows accessing the IBM Quantum systems and simulators.
- [Qiskit IBM Experiment](https://github.com/Qiskit/qiskit-ibm-experiment) - [pypi](https://pypi.org/project/qiskit-ibm-experiment) - This project contains a service that allows accessing the IBM Quantum experiment database.


## Setup

Overview:
- [Account Setup steps](https://github.com/Qiskit/qiskit-ibm-runtime#account-setup) (IBM Cloud or IBM Quantum)
- [IBM Cloud - Getting started instructions](https://cloud.ibm.com/docs/quantum-computing?topic=quantum-computing-get-started)

Steps to access __IBM Quantum__ instance:
- [Get your API key](https://quantum-computing.ibm.com/account) and save it in .env as QISKIT_IBM_TOKEN
- use channel="ibm_quantum" when creating the service

OR - Steps to access __IBM Cloud__ instance:

- [Create a quantum Instance on IBM Cloud](https://cloud.ibm.com/quantum/instances)
- [Create an IBM Cloud API Key](https://cloud.ibm.com/iam/apikeys)
- use channel="ibm_cloud" when creating the service

In [ ]:
ibmcloudinstance="crn:v1:bluemix:public:quantum-computing:us-east:a/0d6f8e0db0a1523f2ca61c2cb55c893b:150e4c08-58e0-43f3-9484-203044277dbf::"

In [ ]:
# Load API Key
import os
from dotenv import load_dotenv
load_dotenv() # take environment variables from .env.
qiskit_api_key = os.environ.get("QISKIT_IBM_TOKEN")
assert qiskit_api_key is not None

In [ ]:
# Connect to ibm quantum service
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum", token=qiskit_api_key)

if False:
    # Optional - save credentials to disk
    try:
        QiskitRuntimeService.save_account(channel="ibm_quantum", token=quiskit_api_key)
    except:
        print("Account info already saved")
    # Then you can login without args
    service = QiskitRuntimeService()

In [ ]:
# display current supported backends
for backend in service.backends():
    print(f"{backend.name},    qubits:{backend.num_qubits},    status:{backend.status().status_msg},   pending_jobs:{backend.status().pending_jobs}")

## Test backend on simulator

In [ ]:
# list available programs
service.pprint_programs()

In [ ]:
# run hello-world program on simulator
program_inputs = {'iterations': 1}
backend = service.least_busy(simulator=True, operational=True)
print(f"least_busy simulator backend: {backend.name}")
options = {"backend": backend.name}
job = service.run(program_id="hello-world",
                  options=options,
                  inputs=program_inputs
                 )
print(f"job id: {job.job_id()}")
result = job.result()
print(result)

[Check jobs on web console](https://quantum-computing.ibm.com/jobs)

## Run on a real quantum device

In [ ]:
# Ask IBM Quantum for its least busy device that isn't a simulator.
backend = service.least_busy(simulator=False, operational=True)
print(f'least_busy real backend: {backend.name}')
print(f"backend info: {backend.name},    qubits:{backend.num_qubits},    status:{backend.status().status_msg},   pending_jobs:{backend.status().pending_jobs}")
print(f"backend coupling_map:{backend.coupling_map}")

In [ ]:
# prepare circuit
from qiskit import QuantumCircuit
bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()
print(bell.draw(output='text'))

In [ ]:
# Run job
from qiskit_ibm_runtime import Session, Options, Sampler
from qiskit.tools.monitor import job_monitor

options = Options(optimization_level=1)
options.execution.shots = 1024  # Options can be set using auto-complete.
print(f"backend: {backend.name}")

session = Session(service=service, backend=backend.name)
sampler = Sampler(session=session, options=options)
job = sampler.run(circuits=bell)
print(f"job_id: {job.job_id()}")

# monitor job submission
job_monitor(job,interval=10)

# wait for results
result = job.result()
print(f"Job result is {result}")

[Check jobs on web console](https://quantum-computing.ibm.com/jobs)

In [ ]:
# count measures
counts = result.quasi_dists[0].binary_probabilities()
print(f"Counts:{counts}")

In [ ]:
# Plot the results
from qiskit.tools.visualization import plot_histogram
plot_histogram(result.quasi_dists[0].binary_probabilities(), title="bell")

## Short version

In [ ]:
# 1. Create a simple quantum program called a 'quantum circuit'.
from qiskit import QuantumCircuit
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, [1, 2])
qc.measure_all()
print(qc.draw())

# 2. Ask IBM Quantum for its least busy device that isn't a simulator.
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler
service = QiskitRuntimeService(channel="ibm_quantum", token=qiskit_api_key)
backend = service.least_busy(simulator=False, operational=True)
print(f'Running on {backend.name}')

# 3. Run your circuit on that device
with Session(backend=backend):
    sampler = Sampler()
    result = sampler.run(qc).result()

# 4. Plot the results
from qiskit.visualization import plot_histogram
distribution = result.quasi_dists[0].binary_probabilities()
plot_histogram(distribution)